In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from src import result2dataframe, plot_horizontal_bar, TextScraper, CounterWithRefactor

In [3]:
URLS = ["https://us.burberry.com/womens-new-arrivals-new-in/",
        "https://us.burberry.com/womens-new-arrivals-new-in/?start=2&pageSize=120&productsOffset=&cellsOffset=8&cellsLimit=&__lang=en"]

URL = "https://www.versace.com/us/en-us/women/new-arrivals/new-in/"

URL_i = "https://us.dolcegabbana.com/en/women/highlights/new-in/?page="

In [4]:
# SCRAPING & CREATING A LIST OF LINKS
doc = []
for url in URLS:
    text_scraper = TextScraper(url)
    doc_i = text_scraper.get_words('a', 'href', '-p80')
    doc += doc_i

# DUPLICATING THE LIST OF LINKS
doc_uniq = set(doc)
print("Number of unique items:" + str(len(doc_uniq)))

# CREATING A DICTIONARY WITH WORDS : COUNTS AND KEY : VALUE PAIRS

counter = CounterWithRefactor(doc_uniq)
result = counter.get_count(replace_strings=["/"],
                           split_char="-",
                           must_start_with=False,
                           split_char_2=False)

df_sorted = result2dataframe(result)
df_rel = df_sorted[df_sorted['counts'] > 3]
print(df_rel.head())
print(df_rel.shape)

# PLOTTING
plot_horizontal_bar(df_rel.index, df_rel['counts'],
                    title="Most used words in Burberry 'New in' SS2020 Women collection",
                    save="SS2020_Burberry_word_frequency.jpg")

df_rel['brand'] = 'burberry'
df_burberry = df_rel

Number of unique items:56
         counts
words          
quilted       4
tie           4
lola          4
small         4
motif         5
(21, 1)


/home/samir/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
text_scraper = TextScraper(URL)
doc = text_scraper.get_words('a', 'href', '/us/en-us/women/new-arrivals/new-in/')

# for url in URLS:


# DEDUPLICATING THE LIST OF LINKS
doc_uniq = set(doc)
print("Number of unique items:" + str(len(doc_uniq)))
# print(doc_uniq)

counter = CounterWithRefactor(doc_uniq)
result = counter.get_count(replace_strings=["/us/en-us/women/new-arrivals/new-in/"],
                           split_char="/",
                           must_start_with="/us/en-us/women/new-arrivals/new-in/?",
                           split_char_2="-")

df_sorted = result2dataframe(result)
df_rel = df_sorted[df_sorted['counts'] > 2]
# print(df_rel.head())
# print(df_rel.shape)

# PLOTTING
plot_horizontal_bar(df_rel.index, df_rel['counts'], color="#FFD700",
                    title="Most used words in Versace 'New in' SS2020 Women collection",
                    save="SS2020_Versace_word_frequency.jpg")

df_rel['brand'] = 'versace'

df_versace = df_rel

Number of unique items:100


/home/samir/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# CREATING LIST OF RELEVANT URLS
URLS_i = []
# URLS = list(URLS)
for i in [1, 2, 3, 4]:
    u = ''.join((URL_i, str(i)))
    URLS_i.append(u)

# print(URLS)

# SCRAPING & CREATING A LIST OF LINKS
doc = []
for url in URLS_i:
    text_scraper = TextScraper(url)
    doc_i = text_scraper.get_words('a', 'aria-label', 'Visit')
    doc += doc_i

# print(doc)

# DEDUPLICATING THE LIST OF LINKS
doc_uniq = set(doc)
print("Number of unique items:" + str(len(doc_uniq)))

counter = CounterWithRefactor(doc_uniq)
result = counter.get_count(replace_strings=["Visit", " product page"],
                           split_char=" ",
                           must_start_with=False,
                           split_char_2=False)

df_sorted = result2dataframe(result)
df_rel = df_sorted[df_sorted['counts'] > 4]
# print(df_rel.head())
# print(df_rel.shape)


# PLOTTING
plot_horizontal_bar(df_rel.index, df_rel['counts'], color="#E0115F",
                    title="Most used words in D&G 'New in' SS2020 Women collection",
                    save="SS2020_D&G_word_frequency.jpg")

df_rel['brand'] = 'd&g'

df_dg = df_rel

Number of unique items:76


/home/samir/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
df_brands = pd.concat([df_versace.reset_index(), df_burberry.reset_index(), df_dg.reset_index()])

df_brands['words'] = df_brands['words'].apply(lambda x: str(x).upper())

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_brands['words'])
print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

def brand_target(brand):
    if not brand in ['versace', 'burberry', 'd&g']:
        raise ValueError(f'Invalid brand: {brand}')
    if 'versace' in brand:
        return 0
    if 'burberry' in brand:
        return 1
    if 'd&g' in brand:
        return 2

[ 8 21 29 24 16 36 25  5 43 38  1  3  7  0 33 20 15 40 44 48 34 44 28 42
 31 51 27 45 19 32  3 46 26 37 10 12 30 38 41 11 33 18 38  4 17 39 35 47
 14 11 23 36  9 52 13  6  2 42 33  3 22 50 49]


In [8]:
brands_transformed = df_brands['brand'].apply(lambda x: brand_target(x))

y = brands_transformed

X = onehot_encoded

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
models = {}

models.update({"Baseline": DummyClassifier(random_state=391488407)})
# models.update({"Support vector (C=1.30)": SVC(C=1.30, gamma="scale")})
# models.update({"Support vector (C=0.45)": SVC(C=0.45, gamma="scale")})
# models.update({"Support vector (C=4.80)": SVC(C=4.80, gamma="scale")})
models.update({"Support vector (C=1.00)": SVC(C=1.00, gamma="scale")})
models.update({"Support vector (C=0.25)": SVC(C=0.25, gamma="scale")})
models.update({"Support vector (C=4.00)": SVC(C=4.00, gamma="scale")})
models.update({"Random Forest": RandomForestClassifier(n_estimators=100, random_state=1283220422)})
models.update({"Logistic Regression (C=1.00)": LogisticRegression(C=1.00, solver="liblinear", penalty="l1")})
models.update({"Logistic Regression (C=0.25)": LogisticRegression(C=0.25, solver="liblinear", penalty="l1")})
models.update({"Logistic Regression (C=4.00)": LogisticRegression(C=4.00, solver="liblinear", penalty="l1")})
models.update({"1-nn euclidean": KNeighborsClassifier(n_neighbors=1)})
models.update({"1-nn cosine": KNeighborsClassifier(n_neighbors=1, metric="cosine")})
models.update({"5-nn cosine": KNeighborsClassifier(n_neighbors=5, metric="cosine")})

In [10]:
for clf_key in models.keys():
    clf = models[clf_key]
    clf.fit(X_train, y_train)

/home/samir/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [11]:
scores = pd.Series({clf_name: models[clf_name].score(X_test, y_test) for clf_name in models.keys()}, 
                   name="Accuracy")

In [12]:
scores

Baseline                        0.285714
Support vector (C=1.00)         0.142857
Support vector (C=0.25)         0.190476
Support vector (C=4.00)         0.142857
Random Forest                   0.142857
Logistic Regression (C=1.00)    0.190476
Logistic Regression (C=0.25)    0.190476
Logistic Regression (C=4.00)    0.142857
1-nn euclidean                  0.142857
1-nn cosine                     0.142857
5-nn cosine                     0.190476
Name: Accuracy, dtype: float64